In [1]:
import pandas as pd
import numpy as np

import requests

import json

from datetime import datetime, timedelta

from pprint import  pprint

import plotly.express as px


In [2]:
lang          = 'es'

start_date1   = '2024-10-15'
end_date1     = '2024-01-01'

start_date2   = '2023-12-31'
end_date2     = '2023-01-01'

start_date3   = '2022-12-31'
end_date3     = '2022-01-01'

In [3]:
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Host": "apidatos.ree.es"
}

In [ ]:
def generacion_datos(lang, start_date1, end_date1, start_date2, end_date2, start_date3, end_date3):

    #Datos generación 2024
    query = f"start_date={end_date1}T00:00&end_date={start_date1}T23:59&time_trunc=day"


    endpoint = f"https://apidatos.ree.es/{lang}/datos/generacion/estructura-generacion?{query}"
    response = requests.get(url= endpoint, headers= headers)
    data = response.json()

    lista_nombres = list()
    lista_tipos = list()
    lista_valores = list()
    lista_porcentajes = list()
    lista_fechas = list()




    for dato in data['included']:
        nombre = dato['attributes']['title']
        tipo = dato['attributes']['type']


        for i in dato['attributes']['values']:
            valor = i['value']

            porcentaje = i['percentage']

            fecha = i['datetime']
            fecha = pd.to_datetime(fecha)


            fecha =  fecha.strftime("%d/%m/%Y")

            lista_nombres.append(nombre)
            lista_tipos.append(tipo)
            lista_valores.append(valor)
            lista_porcentajes.append(porcentaje)
            lista_fechas.append(fecha)


    df_generacion1 = pd.DataFrame()
    df_generacion1['nombre']               = lista_nombres
    df_generacion1['tipo de energía']      = lista_tipos
    df_generacion1['Valores']              = lista_valores
    df_generacion1["Porcentaje"]           = lista_porcentajes
    df_generacion1["Fecha actualización"]  = lista_fechas

    #Datos generación 2023
    query = f"start_date={end_date2}T00:00&end_date={start_date2}T23:59&time_trunc=day"


    endpoint = f"https://apidatos.ree.es/{lang}/datos/generacion/estructura-generacion?{query}"
    response = requests.get(url= endpoint, headers= headers)
    data = response.json()

    lista_nombres = list()
    lista_tipos = list()
    lista_valores = list()
    lista_porcentajes = list()
    lista_fechas = list()




    for dato in data['included']:
        nombre = dato['attributes']['title']
        tipo = dato['attributes']['type']


        for i in dato['attributes']['values']:
            valor = i['value']

            porcentaje = i['percentage']

            fecha = i['datetime']
            fecha = pd.to_datetime(fecha)


            fecha =  fecha.strftime("%d/%m/%Y")

            lista_nombres.append(nombre)
            lista_tipos.append(tipo)
            lista_valores.append(valor)
            lista_porcentajes.append(porcentaje)
            lista_fechas.append(fecha)


    df_generacion2 = pd.DataFrame()
    df_generacion2['nombre']               = lista_nombres
    df_generacion2['tipo de energía']      = lista_tipos
    df_generacion2['Valores']              = lista_valores
    df_generacion2["Porcentaje"]           = lista_porcentajes
    df_generacion2["Fecha actualización"]  = lista_fechas

    #Datos generación 2022
    query = f"start_date={end_date3}T00:00&end_date={start_date3}T23:59&time_trunc=day"


    endpoint = f"https://apidatos.ree.es/{lang}/datos/generacion/estructura-generacion?{query}"
    response = requests.get(url= endpoint, headers= headers)
    data = response.json()

    lista_nombres = list()
    lista_tipos = list()
    lista_valores = list()
    lista_porcentajes = list()
    lista_fechas = list()




    for dato in data['included']:
        nombre = dato['attributes']['title']
        tipo = dato['attributes']['type']


        for i in dato['attributes']['values']:
            valor = i['value']

            porcentaje = i['percentage']

            fecha = i['datetime']
            fecha = pd.to_datetime(fecha)


            fecha =  fecha.strftime("%d/%m/%Y")

            lista_nombres.append(nombre)
            lista_tipos.append(tipo)
            lista_valores.append(valor)
            lista_porcentajes.append(porcentaje)
            lista_fechas.append(fecha)


    df_generacion3 = pd.DataFrame()
    df_generacion3['nombre']               = lista_nombres
    df_generacion3['tipo de energía']      = lista_tipos
    df_generacion3['Valores']              = lista_valores
    df_generacion3["Porcentaje"]           = lista_porcentajes
    df_generacion3["Fecha actualización"]  = lista_fechas

    #Concatenación de generaciones de diferentes años
    df_generacion = pd.concat([df_generacion3, df_generacion2, df_generacion1], ignore_index = True)

    return df_generacion

df_generacion = generacion_datos(lang, start_date1, end_date1, start_date2, end_date2, start_date3, end_date3)


df_generacion

In [5]:
df_generacion.to_csv('generacion_estructura.csv')

In [ ]:
px.line(data_frame = df_generacion, x = 'Fecha actualización', y = 'Valores', color = 'nombre',
        line_group= 'tipo de energía',
        title= 'Tipo de energía',
        markers = True)

In [ ]:
px.box(data_frame = df_generacion,
       x = 'Valores',
       y = 'nombre',
       color= 'nombre')

In [ ]:
px.box(data_frame= df_generacion,
       x = 'Valores',
       y = 'tipo de energía',
       title= 'Box tipo de energía',
       color = 'tipo de energía')

In [ ]:
px.histogram(data_frame= df_generacion,
             x = 'Valores',
             y = 'Porcentaje',
             color = 'nombre',
             nbins= 200)

In [ ]:
px.histogram(data_frame= df_generacion,
             x = 'Valores',
             y = 'Porcentaje',
             color = 'tipo de energía',
             title= 'Histograma por su tipo de energía',
             facet_col= 'tipo de energía',
             nbins= 50)

In [ ]:
px.violin(data_frame=df_generacion,
             x = 'nombre',
             y = 'Valores',
             color = 'nombre',
             points= 'suspectedoutliers'
             )

In [ ]:
px.scatter(data_frame= df_generacion,
           x = 'Valores',
           y = 'Porcentaje',
           color = 'nombre',
           hover_name= 'nombre',
           opacity = 0.1)